## Colab Environment setup

### Env Control parameters

In [2]:
from pathlib import Path

In [3]:
# --- Run setting - colab or local
ENVIRONMENT = 'Colab'

# --- Paths
DRIVE_PROJECT_PATH = Path('/content/drive/MyDrive/mlds_final_project/project')
LOCAL_PROJECT_PATH = Path.cwd().parent

# ---Run name
RUN_NAME = 'yaniv_test'
ADD_TIMESTAMP_TO_FLODER_NAME = False

In [4]:
# --- Mount google drive
if ENVIRONMENT == 'Colab':
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = DRIVE_PROJECT_PATH

else:
    raise ValueError(f'ENVIRONMENT - Only Colab runs are supported here!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Download dataset from Kaggle

In [5]:
data_path = Path('./data')

TRAIN_DATA_PATH = data_path / Path('train/train')
TEST_DATA_PATH = data_path / Path('test/test')

print(f'Train data path on colab: {TRAIN_DATA_PATH.absolute()}')
print(f'Test data path on colab: {TEST_DATA_PATH.absolute()}')


Train data path on colab: /content/data/train/train
Test data path on colab: /content/data/test/test


In [6]:
import shutil

# --- Download data from Kaggle
FORCE_DATA_DOWNLOAD = False

if not ENVIRONMENT == 'Colab':
  raise Exception('Auto data download only works in Colab mode!')

# --- check folder existance
if data_path.exists() and not FORCE_DATA_DOWNLOAD:
  raise Exception('Data already on local, stopping!')

# --- download kaggle token
kaggle_token_path = PROJECT_PATH / Path('kaggle.json')
if not kaggle_token_path.exists():
  raise FileNotFoundException(f'Kaggle key not found in drive: {kaggle_token_path.absolute()}')

Path('/root/.kaggle').mkdir(parents=True, exist_ok=True)
shutil.copyfile(kaggle_token_path, f'/root/.kaggle/{kaggle_token_path.name}')
!chmod 600 /root/.kaggle/kaggle.json
print('Kaggle key copied to local')

# --- Download data
!rm -rf ./data
!mkdir ./data
!pip install kaggle
!kaggle datasets download -d biaiscience/dogs-vs-cats
!unzip dogs-vs-cats.zip -d ./data
!rm dogs-vs-cats.zip


Exception: Data already on local, stopping!

## Project code

The available constants for use after the above code runs:


*   PROJECT_PATH - Project root path
*   TRAIN_DATA_PATH - train data path
*   TEST_DATA_PATH - test data path



### Create current run output folder

In [7]:
from datetime import datetime
from pathlib import Path


def create_output_dir(project_path, run_name: str, add_timestamp: bool) -> str:
    """
    Creates output path based on run name and timestamp and returns path string
    :param project_path: name of project path to create the output folder in
    :param run_name: str: name of run / config
    :param add_timestamp: bool: add timestamp to folder name if true
    :return: output folder path string
    """
    if add_timestamp:
        timestamp_str = datetime.now().strftime("%d_%m_%Y__%H_%M_%S")
        relative_output_dir = Path(f'{run_name}_{timestamp_str}')
    else:
        relative_output_dir = Path(f'run_outputs/{run_name}')

    output_path = Path(project_path) / Path('outputs') / relative_output_dir

    output_path.mkdir(parents=True, exist_ok=True)
    print(f'Run output path: {output_path}')

    return str(output_path)

## Datasets and dataloaders


In [8]:
import glob
from enum import Enum
from pathlib import Path
import pandas as pd
import torch
from PIL import Image

from torch.utils.data import Dataset

class DogsVsCatsLabels(int, Enum):
    CAT = 0
    DOG = 1

class DogsVsCatsDataset(Dataset):
    def __init__(self, images_path: str, transform=None, cache_data=True, shuffle=False):
        self.cache_data = cache_data
        self.transform = transform

        images_path = Path(images_path)
        assert images_path.exists(), f'data path not found: {images_path}'

        file_list = glob.glob(str(images_path / Path('*.jpg')))
        labels_and_ids = [Path(file).name.split('.')[:2] for file in file_list]

        labels = [item[0] for item in labels_and_ids]
        labels = [DogsVsCatsLabels.DOG if label == 'dog' else DogsVsCatsLabels.CAT for label in labels]

        ids = [item[1] for item in labels_and_ids]

        self.metadata = pd.DataFrame()
        self.metadata['id'] = ids
        self.metadata['path'] = file_list
        self.metadata['label'] = labels

        if shuffle:
            self.metadata = self.metadata.sample(frac=1)

        self.cache = {}

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, item):
        item_metadata = self.metadata.iloc[item]

        if item in self.cache:
            image = self.cache[item]
        else:
            image = Image.open(item_metadata['path'])

            if self.transform:
                image = self.transform(image)

            if self.cache_data:
                self.cache[item] = image

        label = item_metadata['label']
        label = torch.Tensor([label])
        return image, label

In [9]:
from torchvision.transforms import transforms


def get_train_transform():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
        transforms.RandomErasing(),
    ])
    return transform

def get_test_transform():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])
    return transform

## Model


In [10]:
import torch
import torchvision


def get_resnet18(pretrained=False, out_features=None, path=None):
    model = torchvision.models.resnet18(pretrained=pretrained)
    if out_features is not None:
        model.fc = torch.nn.Linear(in_features=512, out_features=out_features)
    if path is not None:
        model.load_state_dict(torch.load(path))

    return model

## Runner

In [11]:
 run_name = 'yaniv_test'
 create_output_dir(PROJECT_PATH, RUN_NAME, ADD_TIMESTAMP_TO_FLODER_NAME)


Run output path: /content/drive/MyDrive/mlds_final_project/project/outputs/run_outputs/yaniv_test


'/content/drive/MyDrive/mlds_final_project/project/outputs/run_outputs/yaniv_test'

In [12]:
import numpy as np
import torch
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam, lr_scheduler
from torch.utils.data import DataLoader, random_split
import torch
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam, lr_scheduler
from torch.utils.data import DataLoader, SubsetRandomSampler

train_transform = get_train_transform()
train_val_dataset = DogsVsCatsDataset(TRAIN_DATA_PATH, transform=train_transform, cache_data=False, shuffle=True)
# --- Split train and validation
train_split_ratio = 0.75

train_split_th = int(np.floor(train_split_ratio * len(train_val_dataset)))
train_val_indices = list(range(len(train_val_dataset)))

train_dataset, valid_dataset = random_split(train_val_dataset, [train_split_th, len(train_val_dataset) - train_split_th])

# --- Dataloaders
BATCH_SIZE = 100

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

# --- model, optimizer, loss
model = get_resnet18(pretrained=False, out_features=1)
optimizer = Adam(model.parameters())
criterion = BCEWithLogitsLoss()

# --- train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = model.to(device)
model.train()

NUM_EPOCHS = 10
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0
    num_batches = len(train_dataloader)

    for batch_idx, batch in enumerate(train_dataloader):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        preds = model(inputs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        if batch_idx % 20 == 0:
            print(f'epoch {epoch}, batch {batch_idx}/{num_batches}, loss: {loss}')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Using device: cuda
epoch 0, batch 0/188, loss: 0.6978673934936523
epoch 0, batch 20/188, loss: 0.6475023031234741
epoch 0, batch 40/188, loss: 0.6244397759437561
epoch 0, batch 60/188, loss: 0.6569477915763855
epoch 0, batch 80/188, loss: 0.6624540090560913
epoch 0, batch 100/188, loss: 0.5751601457595825
epoch 0, batch 120/188, loss: 0.6005346775054932
epoch 0, batch 140/188, loss: 0.5722479224205017
epoch 0, batch 160/188, loss: 0.5463268756866455
epoch 0, batch 180/188, loss: 0.5924108028411865
epoch 1, batch 0/188, loss: 0.5652230978012085
epoch 1, batch 20/188, loss: 0.5102185606956482
epoch 1, batch 40/188, loss: 0.4996943473815918
epoch 1, batch 60/188, loss: 0.5917226076126099
epoch 1, batch 80/188, loss: 0.48302435874938965
epoch 1, batch 100/188, loss: 0.48615023493766785
epoch 1, batch 120/188, loss: 0.5472533702850342
epoch 1, batch 140/188, loss: 0.5258176922798157


KeyboardInterrupt: 